In [ ]:
import numpy as np
import pandas as pd
import statistics as st
import math
!pip install numdifftools
import numdifftools as ndt

     |████████████████████████████████| 962kB 9.6MB/s 


In [ ]:
# Loss functions
# https://ml-cheatsheet.readthedocs.io/en/latest/loss_functions.html
# Define a list of loss fns and an array in ERM
#http://www.ws.binghamton.edu/fowler/fowler%20personal%20page/EE522_files/EECE%20522%20Notes_26%20Ch_11B.pdf

In [ ]:
#Preprocessing

def split_data(data, r): 
    train_len = int(len(data) * r) 
    test_len = len(data) - train_len
    test = [] 
    train = list(data) 
    while len(test) < test_len:  
        inx = random.randrange(train_len)  
        test.append(train.pop(inx)) 
    return train, test 

def key_feat_map(data_arr):
  dict1 = {}
  for lst in data_arr:
    k = lst[-1]
    if k in dict1.keys():
      dict1[k].append(lst[:-1])
    else:
      dict1[k] = [np.asarray(lst[:-1])]
  for k in dict1:
    dict1[k] = np.asarray(dict1[k])
    #print(dict1[k].shape)
  return dict1

In [ ]:
#define precision, recall, fmeasure, accuracy, confusion matrix etc

def confusion_matrx(out_true_dict,num_class):
  conf_m = np.zeros(dtype=int,shape=(num_class,num_class))
  
  for i in out_true_dict.keys():
    true_v = i
    pred_vs = out_true_dict[i]
    for pred_v in pred_vs:
      conf_m[true_v][pred_v] +=1

  acc = 0
  for i in range(num_class):
    acc += conf_m[i]
  acc = acc/sum(sum(conf_m))
  return conf_m

def precis_recall_f(out_label,true_label,num_class):
  cfm = confusion_matrx(out_label,true_label,2)
  precision = (1/(cfm[1][1] + cfm[0][1]))*cfm[1][1]
  recall = (1/(cfm[0][0] + cfm[1][0]))*cfm[0][0]
  fscore = 2*precision*recall*(1/(precision+recall))
  return precision,recall,fscore

#http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.104.8244&rep=rep1&type=pdf
#https://people.eecs.berkeley.edu/~jordan/courses/260-spring10/other-readings/chapter13.pdf
#https://dl.acm.org/doi/10.1023/A%3A1010920819831
#http://mulan.sourceforge.net/


In [ ]:
x = np.ones(shape=(3,4))
x[1][0] = 2.5
x[1][2] = 3
x[0][1] = 4.5
x[0][3] = 5.6
print(x)
mu,sd=MLE_Gaussian_Params(x)
PDF_normaldistr(x[0][:-1],mu,sd)

[[1.  4.5 1.  5.6]
 [2.5 1.  3.  1. ]
 [1.  1.  1.  1. ]]
4


ValueError: ignored

In [ ]:
#MLE with Gaussian || Assumptions: class conditional density is gaussian, priors all equiprobable || Bayes classification
#MLE => Loss function is KL-Divergence
#http://www.cs.columbia.edu/~mcollins/em.pdf

def MLE_Gaussian_Params(data_lst):
  n = data_lst.shape[0] #number of data points
  k = data_lst.shape[1] #feature vector dimension
  mu = data_lst[0][:]
  for j in range(1,n):
    mu = mu + data_lst[j][:]
  mu = mu/n
  
  sd = np.zeros(shape=(k,k),dtype=float)
  print(k)
  for j in range(n):
    tmp = data_lst[j][:] - mu
    #print(np.transpose(tmp[np.newaxis]).shape)
    sd += np.matmul(np.transpose(tmp[np.newaxis]),tmp[np.newaxis])
  
  sd = sd/n
  
  return mu,sd

def PDF_normaldistr(inpt,mu,sd):
  #mu = lst[0]
  #sd = lst[1]
  tmp = inpt-mu
  tmp = tmp[np.newaxis]
  k = tmp.shape[1]
  sdi = np.linalg.inv(sd)
  expt = np.matmul(tmp,sdi)
  expt = np.matmul(expt,np.transpose(tmp))*(-0.5)
  expt = math.exp(expt[0][0])

  return expt*((2*math.pi)**(-0.5*k))*((np.linalg.det(sd))**(-0.5))
  #print(expt)

def predict_label_MLEGauss(lst,lst_params,prior_arr = 1):
  out_lab = (-1,0)

  for indx in len(lst_params):
    theta = lst_params[indx]
    fx_y = PDF_normaldistr(lst,theta[0],theta[1])
    fx_y = prior*fx_y
    if(out_lab[0]==-1):
      out_lab = (fx_y,indx)
    else:
      if(out_lab[0]<fx_y):
        out_lab = (fx_y,indx)

  return out_lab[1]

def Classify_MLE_Gauss(train_data_dict,test_data_dict):
  lst_params = []
  #true_class = []
  for k in train_data_dict.keys():
    lst_d = train_data_dict[k]
    mu,sd = MLE_Gaussian_Params(lst_d)
    lst_params.append(tuple(mu,sd))
    true_class.append(k)

  dict_true_out = {}
  for ky in test_data_dict.keys():
    lst_d = test_data_dict[ky]
    out_list = []
    for arr in lst_d:
      out_l = predict_label_MLEGauss(arr,lst_params)
      out_list.append(out_l)
    
    dict_true_out[ky] = out_list
    
  return dict_true_out


In [ ]:
#MLE for Naive Bayes || 

def MLE_uniGauss(data):
  #data has to be single feature ka array
  mu = np.sum(data)
  mu = mu/data.shape[0]
  
  tmp = [(t - mu)^2 for t in data]
  sd = (1/(data.shape[0]))*(sum(tmp))
  return [mu,sqrt(sd)]

def pdf_uniGauss(x,lst):
  mu = lst[0]
  sd = lst[1]
  expt = (-0.5)*(x-mu)*(x-mu)*(1/sd)
  return (1/sd)*(0.5)*(1/(math.pi))*math.exp(expt)

def MLE_uniExp(data):
  #data has to be single feature ka array
  #would need to be modified to Laplacian agar neg features possible
  mu = np.sum(data)
  mu = mu/data.shape[0]
  return [mu]

def pdf_uniExp(x,mu1):
  mu = mu1[0]
  return (1/mu)*(math.exp(-x/mu))

def MLE_uniUnf(data):
  #data has to be single feature ka array
  return [np.amax(data)]

def pdf_uniUnf(x,mu):
  return 1/mu[0]

def MLE_predictLab_Naive(test,params_arr,fn_arr):
  #all_vals = []
  max_val = (-1,0)
  for key in params_arr.keys():
    params = params_arr[key]
    val = 1
    for i in len(range(test)):
      pxi_y = fn_arr[i](test[i],params[i])
      val = val*pxi_y
    py = 1 #equiprobable for now
    val = val*py
    if(max_val[0]==-1):
      max_val = (val,key)
    else:
      if(max_val[0]<val):
        max_val = (val,key)

  #  all_vals.append(tuple([val,key]))
  print("boo") 
  return max_val[1] 

def MLE_classify_Naive(train_dict,test_dict,MLE_arr,fn_arr,is_iid):
  
  #training
  params_arr = {}
  for key in train_dict.keys():
    lst_d = train_dict[key]
    len_feat = len(lst_d)
    feat_params = [0 for t in range(len_feat)]
    if(is_iid):
      for i in range(len_feat):
        xi = lst_d[:,i]
        feat_params[i] = MLE_arr[i](xi)
      print("fin if")
    else:
      #build dict
      MLE_dict = {}
      for i in range(len(MLE_arr)):
        key = MLE_arr[i]
        if key in MLE_dict.keys():
          MLE_dict[key].append(i)
        else:
          MLE_dict[key] = [i]
      
      for fn in MLE_dict.keys():
        feats = MLE_dict[fn]
        data_all = lst_d[feats[0]]
        for j in range(1,len(feats)):
          np.append(data_all,lst_d[feats[j]])
        lst_params = fn(data_all)
        for t in feats:
          feat_params[t] = lst_params
      print("fin els")

    params_arr[key] = feat_params  

    #testing
    dict_true_out = {}
    for k in test_dict.keys():
      lst_feats = test_dict[k]
      out_labels = []
      for feat in lst_feats:
        lbl = MLE_predictLab_Naive(feat,params_arr,fn_arr)
        out_labels.append(lbl)
      dict_true_out[k] = out_labels
    return dict_true_out

In [ ]:
x = np.ones(shape=(3,10))
np.cov(x)

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [ ]:
#MLE FOR GMMs
#Expectation minimization algorithm

def latent_probs(num_data,data,gmm_coeffs,gmm_mu,gmm_sd):
  gamma_wts = np.zeros(shape=(num_data,len(gmm_coeffs)))
  #normal_distrn = [,pdf_uniGauss]
  #all_j_sums = []
  for i in range(num_data):
    temp_sum = 0
    for j in range(len(gmm_coeffs)):
      val = pdf_NormalDistr(data[i],gmm_mu[j],gmm_sd[j])
      gamma_wts[i][j] = val*gmm_coeffs[j]
      temp_sum += val*gmm_coeffs[j]
    #all_j_sums.append(temp_sum)
    gamma_wts[i,:] = gamma_wts[i,:]/temp_sum

  return gamma_wts 

def find_maximized(data,gamma_wts):
  gmm_mu = []
  gmm_coeffs = []
  gmm_sd = []
  n = gamma_wts.shape[0]
  
  for k in range(gamma_wts.shape[1]):
    pi_k = (1/n)*(np.sum(gamma_wts[:,k]))
    mu_k = np.zeroes(shape=len(data[0]))
    for i in range(n):
      mu_k += [gamma_wts[i][k]*t for t in data[i]] ##might give bt with data[i] not iterable
    mu_k = mu_k/(n*pi_k)

    sd_k = np.zeroes(shape=(len(data[0]),len(data[0])))
    for j in range(n):
      tmp = data_lst[j] - mu_k
      sd_k += np.multiply(np.matmul(np.transpose(tmp[np.newaxis]),tmp[np.newaxis]),gamma_wts[j][k])
    
    sd_k = sd_k/(n*pi_k)
    
    gmm_coeffs.append(pi_k)
    gmm_mu.append(mu_k)
    gmm_sd.append(sd_k)

  return gmm_coeffs,gmm_mu,gmm_sd

def logLikeli_EMM(data,gmm_coeffs,gmm_mu,gmm_sd,gamma_coeffs):
  logl = 0
  for i in range(len(data)):
    for j in range(len(gmm_coeffs)):
      val = math.log(gmm_coeffs[j]) + math.log(pdf_NormalDistr(data[i],gmm_mu[j],gmm_sd[j]))
      val = val*gamma_coeffs[i][j]
      logl+=val
  return logl
  print("bpp")

def EM_GMMs(n,data_lst,init_coeff,init_mu,init_sd,tolr=10^-5):
  stopCon=False
  
  init_logL,temp_logL = 0,0

  iter = 0
  while(stopCon==False):
    init_gm_wts = latent_probs(n,data_lst,init_coeff,init_mu,init_sd)
    gmmc,gmmm,gmmsd = find_maximized(data_lst,init_gm_wts)
    #init_gm_wts = latent_probs(n,data_lst,gmc,gmmm,gmmsd)
    init_logL = temp_logL
    temp_logL = logLikeli_EMM(data_lst,gmmc,gmmm,gmmsd,init_gm_wts)

    iter+=1
    if(abs(init_logL-templogL)<tolr and iter>1):
      stopCon = True

  print("bpp")
  return gmmc,gmmm,gmmsd

#https://towardsdatascience.com/gaussian-mixture-models-explained-6986aaf5a95

def pdf_GMM(inpt,gmm_c,gmm_m,gmm_sd):
  val = 0
  for t in range(gmm_c):
    val += gmm_c[t]*pdf_NormalDistr(inpt,gmm_m[t],gmm_sd[t])
  return val

def MLE_predictLab_GMM(inpt,params_arr):
  #all_vals = []
  max_val = (-1,0)
  for key in params_arr.keys():
    params = params_arr[key]
    val = pdf_GMM(inpt,params[0],params[1],params[2])
    py = 1 #equiprobable for now
    val = val*py
    if(max_val[0]==-1):
      max_val = (val,key)
    else:
      if(max_val[0]<val):
        max_val = (val,key)

  #print("boo") 
  return max_val[1] 

def MLE_Bayes_GMM(train_dict,test_dict,init_guess):
  params_dict = {}
  
  for k in train_dict.keys():
    c,mu,sd = EM_GMMs(len(train_dict[k]),train_dict[k],init_guess[0],init_guess[1],init_guess[2])
    params_dict[k] = (c,mu,sd)
  
  tested_labels = {}
  for k in test_dict.keys():
    feats = test_dict[k]
    out_labs = []
    for feat in feats:
      labl = MLE_predictLab_GMM(feat,params_dict)
      out_labs.append(labl)
    tested_labels[k] = out_labs
  
  return tested_labels

http://www.music.mcgill.ca/~ich/classes/mumt611_07/classifiers/ComparisonParzenStudy.pdf
https://www.projectrhea.org/rhea/index.php/Lecture_16_-_Parzen_Window_Method_and_K-nearest_Neighbor_Density_Estimate_OldKiwi
https://shapeofdata.wordpress.com/2013/07/23/gaussian-kernels/
https://shapeofdata.wordpress.com/2013/04/23/nearest-neighbors-classification/
https://stephens999.github.io/fiveMinuteStats/intro_to_em.html
https://www.kth.se/social/upload/541152c5f27654548153cba3/04-problearn-2x2.pdf
https://www.sciencedirect.com/science/article/pii/S0885064X08000526
https://sebastianraschka.com/Articles/2014_kernel_density_est.html#23-the-window-function
http://www.cs.cmu.edu/~aarti/Class/10701_Spring14/slides/MLE_MAP_Part1.pdf


In [ ]:
#MAP of argmax f(X|Y,P)*f(P) || With Naive Bayes

def MAP_expo_fXY_theta(data,theta_prior_param):
  n = len(data)
  return 1/((theta_prior_param)+sum(data))

def MAP_gauss_fXY_theta_varfix(data,theta_prior_params,var):
  n = len(data)
  s = sum(data)
  sm2 = theta_prior_params[1]**2
  return (sm2*s + (var**2)*theta_prior_params[0])/(sm2*n+(var**2))



In [ ]:
#https://ml-cheatsheet.readthedocs.io/en/latest/gradient_descent.html
#https://numdifftools.readthedocs.io/en/latest/reference/generated/numdifftools.core.Gradient.html

def grad_descent(fn,lr,init_guess,maxiter):
  #opt_params = []
  grad = ndt.Gradient(fn)
  tmp_guess1 = init_guess
  update_fn = lambda tmp: tmp - lr*grad(tmp)
  stopCon = False
  iter = 0
  while(stopCon==False):
    tmp_guess1 = update_fn(tmp_guess1)
    iter+=1
    if(iter>maxiter):
      stopCon = True
  return tmp_guess1

In [ ]:
#Parzen window est. 
def hypercube_fn(subs_vec,width=1):
  for t in subs_vec:
    if(np.abs(t)>(0.5*width)):
      return 0
  return 1
  
def gauss_kernel(subs_vec,width=1):
  d = len(subs_vec)
  expt = np.matmul(subs_vec[newaxis],np.transpose(subs_vec[newaxis]))
  
  return (width**d)*((math.pi)**(-0.5*d))*math.exp(expt*(-0.5/(width**2)))

def parzen_label_est(inpt,train_dict,kernel,width=1):
  #prior equiprobable
  x=0
  px_y = (-1,0)
  for k in train_dict.keys():
    train_pts = train_dict[k]
    tmp_val = 0
    for pt in train_pts:
      in_vec = inpt - pt
      tmp_val += kernel(in_vec,width)
    tmp_val = tmp_val/(n*(width**d))
    if(px_y[0]<tmp_val):
      px_y = (tmp_val,k)
  return px_y[1]

def parzen_classify(train_dict,test_dict,kernel,width):
  test_labs_dict = {}

  for k in test_dict.keys():
    datapts = test_dict[k]
    out_lab = []
    
    for t in datapts:
      out_lab.append(parzen_label_est(t,train_dict,kernel,width))

    test_labs_dict[k] = out_lab
  return test_labs_dict

#https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc
#https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-classification-in-python/
#http://www.svcl.ucsd.edu/courses/ece271A/handouts/BayesIntro.pdf
#file:///D:/!Sem5/ML/assignments/409_2020_Assignment_1.pdf

In [ ]:
x = np.zeros(shape=(2,))
y = np.append(x,x)
print(y)
y = np.append(y,x)
print(y)

[0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0.]


In [ ]:
x = [pdf_uniGauss,pdf_uniUnf,pdf_uniExp]
#x[0]([1,2,3])
dict1 = {}
for i in x:
  dict1[i] = 1
print(dict1)

{<function pdf_uniGauss at 0x7f708d3da400>: 1, <function pdf_uniUnf at 0x7f708d3da6a8>: 1, <function pdf_uniExp at 0x7f708d598c80>: 1}


In [ ]:
# https://static1.squarespace.com/static/5ae0d0b48ab7227d232c2bea/t/5de5b403ae38e01033f5cf38/1575334918495/mle.pdf
# http://www.cs.columbia.edu/~verma/classes/ml/index.html
# https://stats.stackexchange.com/questions/112451/maximum-likelihood-estimation-mle-in-layman-terms
# https://ruder.io/optimizing-gradient-descent/index.html#gradientdescentvariants

In [ ]:
#MLE with GMMs || Assumptions: class conditional density is convex combination, priors all equiprobable || Naive Bayes assumption holds??
#MLE => Loss function is KL-Divergence
#this to be ignored
'''
def MLE_GMM_Params(data_lst):
  n = data_lst.shape[0] #number of data points
  k = data_lst.shape[1] #feature vector dimension
  mu = data_lst[0][:]
  for j in range(1,n):
    mu = mu + data_lst[j][:]
  mu = mu/n
  
  sd = np.zeros(shape=(k,k),dtype=float)
  print(k)
  for j in range(n):
    tmp = data_lst[j][:] - mu
    #print(np.transpose(tmp[np.newaxis]).shape)
    sd += np.matmul(np.transpose(tmp[np.newaxis]),tmp[np.newaxis])
  
  sd = sd/n
  
  return mu,sd
'''